In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontManager


## Loading Machine Learning libraries
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer


from mpl_toolkits.mplot3d import Axes3D

pd.set_option("display.max_rows", 8)
pd.options.mode.chained_assignment = None

%matplotlib inline
%config InlineBacked.figure_format = "retina"

In [4]:
stopwords = pd.read_csv("../data/stopwords.txt", 
                       header = None, 
                       names = ['Stopwords'])

df = pd.read_csv("../data/corp_text.csv")
df.head(5)

,Unnamed: 0,shixin_id,duty
0,1,2905,"被告上海纺友贸易有限公司应于本判决生效后十日内支付原告江苏盛虹面料有限公司货款人民币1,13..."
1,6,3650,"一、被告上海栩瑞电气设备有限公司确认尚欠原告上海宝临超亚电器有限公司货款302,725元，原..."
2,16,11617,原告上海玖峰数码科技有限公司于本判决生效之日起十日内支付被告张宁顺违法解除劳动合同赔偿金93...
3,21,15642,被执行人于2013年4月25前向申请执行人支付工资人民币5万元；被执行人于2013年5月15...
4,22,16229,一、原告上海顶天钢铁发展有限公司与被告内蒙古兴托物资有限公司于2007年1月11日签订的《内...


In [5]:
import jieba ## Chinese Langauge Segementation tool;

In [6]:
list(jieba.cut(df.duty[3], cut_all=True))  ## test for iname;

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/ys/98s0q77d0bl4mpqsp4lf7c740000gn/T/jieba.cache
Loading model cost 0.768 seconds.
Prefix dict has been built successfully.


['被执行人',
 '执行',
 '行人',
 '于',
 '2013',
 '年',
 '4',
 '月',
 '25',
 '前',
 '向',
 '申请',
 '执行',
 '行人',
 '支付',
 '工资',
 '人民',
 '人民币',
 '5',
 '万元',
 '；',
 '被执行人',
 '执行',
 '行人',
 '于',
 '2013',
 '年',
 '5',
 '月',
 '15',
 '前',
 '向',
 '申请',
 '执行',
 '行人',
 '支付',
 '工资',
 '人民',
 '人民币',
 '5',
 '万元',
 '；',
 '上述',
 '款项',
 '支付',
 '至',
 '申请',
 '执行',
 '行人',
 '原',
 '工资',
 '工资卡',
 '账户',
 '（',
 '汇丰',
 '汇丰银行',
 '银行',
 '）；',
 '如',
 '被执行人',
 '执行',
 '行人',
 '于',
 '2013',
 '年',
 '4',
 '月',
 '30',
 '日',
 '之前',
 '未',
 '支付',
 '第一',
 '第一笔',
 '一笔',
 '款项',
 '，',
 '申请',
 '执行',
 '行人',
 '可以',
 '就',
 '所有',
 '剩余',
 '余款',
 '款项',
 '一并',
 '申请',
 '强制',
 '强制执行',
 '执行',
 '；',
 '申请',
 '执行',
 '行人',
 '放弃',
 '其他',
 '仲裁',
 '请求',
 '。']

In [7]:
row,col = df.shape

df['cutword'] = 'cutword'
for i in np.arange(row):
    cutwords = list(jieba.cut(df.duty[i], cut_all=True))
    cutwords = pd.Series(cutwords)[pd.Series(cutwords).apply(len)>1]
    df.cutword[i] = cutwords.values

In [37]:
df.cutword[1:5]

1    [被告, 上海, 电气, 电气设备, 设备, 备有, 有限, 有限公司, 公司, 确认, 原...
2    [原告, 上海, 数码, 数码科技, 科技, 有限, 有限公司, 公司, 判决, 生效, 之...
3    [被执行人, 执行, 行人, 月前, 申请, 执行, 行人, 支付, 工资, 人民, 人民币...
4    [原告, 上海, 顶天, 钢铁, 发展, 有限, 有限公司, 公司, 被告, 内蒙, 内蒙古...
Name: cutword, dtype: object

In [8]:
import gensim
from gensim.corpora import Dictionary
from gensim.models.ldamodel import LdaModel

/usr/local/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [9]:
dictionary = Dictionary(df.cutword)
corpus = [dictionary.doc2bow(word) for word in df.cutword]

In [20]:
lda = LdaModel(corpus=corpus, id2word = dictionary, num_topics = 20, random_state=12)

In [26]:
lda.print_topic(1, topn = 40)

'0.066*"公司" + 0.066*"有限" + 0.065*"有限公司" + 0.064*"上海" + 0.038*"原告" + 0.037*"被告" + 0.030*"管理" + 0.018*"之日" + 0.018*"支付" + 0.016*"判决" + 0.015*"生效" + 0.014*"之日起" + 0.014*"反诉" + 0.013*"十日" + 0.011*"应于" + 0.010*"租赁" + 0.010*"返还" + 0.009*"租金" + 0.008*"房屋" + 0.008*"合同" + 0.007*"投资" + 0.007*"2016" + 0.007*"解除" + 0.007*"签订" + 0.006*"资产" + 0.006*"餐饮" + 0.005*"发展" + 0.005*"人民" + 0.005*"实业" + 0.004*"物业" + 0.004*"企业" + 0.004*"酒店" + 0.004*"经营" + 0.004*"000" + 0.004*"石油" + 0.004*"还原" + 0.004*"使用" + 0.004*"）；" + 0.004*"违约金" + 0.003*"用费"'

In [31]:
for topic in lda.print_topics(num_topics=20, num_words = 5):
    print(topic[1])

0.059*"科技" + 0.037*"公司" + 0.034*"有限公司" + 0.034*"有限" + 0.033*"上海"
0.066*"公司" + 0.066*"有限" + 0.065*"有限公司" + 0.064*"上海" + 0.038*"原告"
0.048*"上海" + 0.032*"有限公司" + 0.031*"有限" + 0.030*"公司" + 0.029*"松江"
0.116*"执行" + 0.097*"行人" + 0.083*"支付" + 0.076*"被执行人" + 0.061*"利息"
0.047*"有限" + 0.045*"被告" + 0.045*"公司" + 0.044*"有限公司" + 0.028*"判决"
0.031*"原告" + 0.030*"物流" + 0.030*"公司" + 0.029*"被告" + 0.028*"判决"
0.096*"浦东" + 0.085*"海市" + 0.085*"上海市" + 0.074*"新区" + 0.074*"浦东新区"
0.058*"2017" + 0.046*"被告" + 0.043*"支付" + 0.040*"原告" + 0.030*"日前"
0.137*"申请" + 0.129*"申请人" + 0.068*"被申请人" + 0.060*"支付" + 0.036*"工资"
0.037*"包装" + 0.033*"被告" + 0.031*"经营部" + 0.031*"营部" + 0.026*"有限"
0.061*"公司" + 0.060*"有限公司" + 0.057*"有限" + 0.043*"上海" + 0.026*"被告"
0.068*"上海" + 0.064*"公司" + 0.064*"被告" + 0.062*"有限公司" + 0.062*"有限"
0.057*"公司" + 0.054*"有限" + 0.050*"有限公司" + 0.030*"被告" + 0.028*"上海"
0.068*"有限" + 0.068*"公司" + 0.062*"有限公司" + 0.053*"上海" + 0.047*"被告"
0.040*"有限" + 0.037*"公司" + 0.034*"有限公司" + 0.030*"上海" + 0.028*"同期"
0.040*"调解" + 0.036*"股权" + 

In [14]:
import pyLDAvis
import pyLDAvis.gensim 

In [15]:

# feed the LDA model into the pyLDAvis instance
lda_viz = pyLDAvis.gensim.prepare(lda , corpus=corpus, dictionary=dictionary)

/usr/local/lib/python3.7/site-packages/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/usr/local/lib/python3.7/site-packages/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/usr/local/lib/python3.7/site-packages/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/usr/local/lib/python3.7/site-packages/joblib/numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
